In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
import math 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [2]:
test = pd.read_csv('https://raw.githubusercontent.com/cicbeast/PythonCoding/main/KaggleComps/titanic/test.csv', index_col=0)
train = pd.read_csv("https://raw.githubusercontent.com/cicbeast/PythonCoding/main/KaggleComps/titanic/train.csv", index_col=0)

In [3]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [6]:
# From the basic info call I know that Cabin will not be a feature that I can reasonably use.
# Next, a full pandas profile

In [7]:
# !pip install pandas-profiling

In [8]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train)
# profile.to_file("titanic_train.html")

In [9]:
# Let's go for a baseline prediction
# Based on 'Women and Children first', let's use those parameters to
# predict survival

In [10]:
len(train[train['Sex'] == 'female'])

314

In [11]:
len(train[train['Age'] <= 15])

83

In [12]:
# These numbers seem reasonable so lets take a look
titanic_women = train[train['Sex'] == 'female']
titanic_children = train[train['Age'] <= 15]

titanic_women.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [13]:
titanic_children.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S
17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q


In [14]:
# Now lets get a women and children df without duplicates
women_and_children = pd.concat([titanic_women,titanic_children]).drop_duplicates().reset_index(drop=True)
women_and_children.shape

(354, 11)

In [15]:
# Our prediction, based on "Women and Children first" would be that
# these 354 people from the titanic survived 
# Let's see how we did with the training set
base = len(women_and_children[women_and_children['Survived'] == 1])
surv = len(train[train['Survived'] == 1])
base/surv

0.7426900584795322

In [16]:
# Our baseline prediction got 74 percent of the survivors in the 
# training set. Not bad. Let's make our first submission using these
# parameters
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [17]:
testsub = test.copy()
testsub.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
# testsub.loc[testsub['Sex'] == "female", 'Survived'] = 1
# testsub.loc[testsub['Age'] <= 15, 'Survived'] = 1

# testsub.head()

In [19]:
# sub = testsub[['Survived']]
# sub.reset_index(level=0, inplace=True)

# sub.head()


In [20]:
# sub.to_csv('sub.csv', index=False)

In [21]:
# Our baseline gives us a 75% Accuracy. Not bad at all
# Now we will try to beat this baseline using modeling
# First we'll need to install/import the required modules
# !pip install category_encoders

In [22]:
# Let's do some feature engineering before we do the train test split
# I want to create a new 'Child' feature, based on age
train.loc[train['Age'] <= 15, 'Child'] = 1
train.loc[train['Age'] > 15, 'Child'] = 0

train.sample(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
PassengerId,,,,,,,,,,,,
639,0,3,"Panula, Mrs. Juha (Maria Emilia Ojala)",female,41.0,0,5,3101295,39.6875,NaN,S,0.0
738,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C,0.0
125,0,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S,0.0
808,0,3,"Pettersson, Miss. Ellen Natalia",female,18.0,0,0,347087,7.7750,NaN,S,0.0
444,1,2,"Reynaldo, Ms. Encarnacion",female,28.0,0,0,230434,13.0000,NaN,S,0.0


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
 11  Child     714 non-null    float64
dtypes: float64(3), int64(4), object(5)
memory usage: 90.5+ KB


In [24]:
# Now we'll clean up our new Child feature
train['Child'].fillna(0, inplace=True)

train['Child'] = train['Child'].astype('int64')

train.sample(6)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
PassengerId,,,,,,,,,,,,
568,0,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.0750,NaN,S,0
876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C,1
548,1,2,"Padro y Manent, Mr. Julian",male,NaN,0,0,SC/PARIS 2146,13.8625,NaN,C,0
417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",female,34.0,1,1,28220,32.5000,NaN,S,0
455,0,3,"Peduzzi, Mr. Joseph",male,NaN,0,0,A/5 2817,8.0500,NaN,S,0
412,0,3,"Hart, Mr. Henry",male,NaN,0,0,394140,6.8583,NaN,Q,0


In [25]:
train.sample(15)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
PassengerId,,,,,,,,,,,,
363,0,3,"Barbara, Mrs. (Catherine David)",female,45.00,0,1,2691,14.4542,NaN,C,0
245,0,3,"Attalah, Mr. Sleiman",male,30.00,0,0,2694,7.2250,NaN,C,0
721,1,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.00,0,1,248727,33.0000,NaN,S,1
509,0,3,"Olsen, Mr. Henry Margido",male,28.00,0,0,C 4001,22.5250,NaN,S,0
194,1,2,"Navratil, Master. Michel M",male,3.00,1,1,230080,26.0000,F2,S,1
240,0,2,"Hunt, Mr. George Henry",male,33.00,0,0,SCO/W 1585,12.2750,NaN,S,0
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.1333,NaN,S,0
305,0,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.0500,NaN,S,0
37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C,0


In [26]:
len(train[train['Child'] == 1])

83

In [27]:
# I know that young men held the 'Master' prefix, so I can use that also
train.loc[train['Name'].str.contains("Master"), 'Child'] = 1

len(train[train['Child'] == 1])


87

In [28]:
# Now let's do our train(val)-test split
xtrain_features = train.drop(columns=['Survived'])
ytrain_features = train['Survived']

xtrain_features.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [29]:
ytrain_features.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [30]:
xtrain_features['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [31]:
xtrain_features['Fare'].nunique()

248

In [32]:
xtrain_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
 10  Child     891 non-null    int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [33]:
xtrain_features = xtrain_features.drop(columns=['Embarked', 'Cabin'])

In [34]:
xtrain_features.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Child
PassengerId,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,0
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,0
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0


In [36]:
xtrain_features['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [40]:
# Let's make a copy of xtrain_features so we can work with it
xtraincopy = xtrain_features.copy()

In [39]:
# We can consider scaling the Fare feature.
scaler = StandardScaler()
scaled = scaler.fit_transform(xtrain_features)

scaled['Fare'].describe()

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(xtrain_features, ytrain_features, test_size=0.25, random_state=42, stratify=ytrain_features)

In [ ]:
print(xtrain.shape, xval.shape, ytrain.shape, yval.shape)

In [ ]:
# Our next step would be to encode 'Sex and Embarked'
encoder = ce.OneHotEncoder(use_cat_names=True)
xtrain_encoded = encoder.fit_transform(xtrain)

In [ ]:
xtrain_encoded.head()

In [ ]:
# Not sure if we like this

In [ ]:
xtrain.sample(6)
 7   Fare      668 non-null    float64

In [ ]:
xval.sample(5)

In [ ]:
ytrain.sample(6)

In [ ]:
yval.sample(5)

In [ ]:
xtrain.info()